# 3.2.1.- Filtrado Colaborativo: *Matrix Factorization*


* En esta notebook vamos a ver la ***técnica de Factorización Matricial (Matrix Factorization) aplicado a los sistemas de recomendación basados en filtrado colaborativo***.


* Veremos los siguientes puntos:
<span></span><br>
    1. [Matrix Factorization](#M1)
<span></span><br>
    2. [Probabilistic Matrix Factorization: Cálculo de las Matrices P y Q](#M2)
<span></span><br>
    3. [Probabilistic Matrix Factorization: Cálculo de las Matrices P y Q, añadiendo los bias](#M3)
<span></span><br>
    4. [Implementación y Ejemplo: PMF Cálculo de las Matrices P y Q](#M4)
<span></span><br>
    5. [Implementación y Ejemplo: PMF Cálculo de las Matrices P y Q, añadiendo los bias](#M5)

<hr>


## <a name="M1">1.- Matrix Factorization</a>

   
* Durante los inicios del filtrado colaborativo el algoritmo del KNN era el más empleado debido a los buenos resultados que reportaba y a la facilidad con la que podían explicarse sus recomendaciones. Sin embargo, este algoritmo tiene una gran desventaja: su escalabilidad. 


* El algoritmo de KNN funciona bien para datasets de tamaño medio, pero, a medida que el dataset crece, los tiempos de cómputo para obtener las recomendaciones se vuelven inasumibles. Aumentar el número de usuarios y/o el número de items implicar ralentizar el cálculo de las similaridades, la búsqueda de los k vecinos y el número de predicciones a realizar.


* Como consecuencia de estos problema y del gran empuje que supuso el [Netflix Prize](https://www.netflixprize.com/) (concurso que ofrecía una recompensa de 1M de dólares al equipo que consiguiera mejorar el RMSE en el dataset de Netflix) comenzaron a ganar fuerza los sistemas de filtrado colaborativo basados en modelos, más concretamente los basados en modelos de factorización matricial.


* El **filtrado colaborativo basado en factorización matricial** se basa en la siguiente idea: las votaciones que los usuarios realizan a los items están condicionadas por una serie de factores latentes intrínsecos a los usuarios y los items. 


* Ilustremos esto con un ejemplo en un sistema de recomendación de películas: Lo que postula la factorización matricial es que los usuarios votan las películas basándose no sólo en la propia película, sino que lo hacen basándose en las características que describen esa película. Si a un usuario le gustan las películas de acción con un toque de comedia, es muy probable que le gusten todas las películas de acción con un toque de comedia. Los algoritmos de filtrado colaborativo buscan estas propiedades intrínsecas al dominio en el que se realizan las recomendaciones y las denominan **factores latentes** u ocultos. 


* Es importante resaltar que estos factores son ocultos, y aunque en el ejemplo de la recomendación de películas podamos suponer que se trata de géneros de cine, el modelo nunca nos va a indicar con qué género se corresponde cada factor.


* Matemáticamente, la factorización matricial consiste en encontrar las matrices $P$ y $Q$ que satisfagan la siguiente expresión:


$$R \approx P \cdot Q$$


* Donde:

    - $R$ representa la matriz (dispersa) con las votaciones de los usuarios (filas) a los items (columnas).
    - $P$ representa las matriz (densa) de factores de los usuarios (filas) con los *k* factores latentes (columnas).
    - $Q$ representa las matriz (densa) de factores de los items (columnas) con los *k* factores latentes (filas).


* A modo de ejemplo podemos ver como a partir de una matriz de votos con notas de 1 a 10, obtenemos para cada usuario y cada item (serie) dos factores latentes ($K=2$) indicados en las matrices $P$ y $Q$ respectivamente:


<img src="./../imgs/03_02_01_01_CF_mf.png" style="width: 600px;"/>


* Basandonos en la idea de que cada factor latente representa una característica que describe al item, podemos "suponer" que los valores de los factores latentes describirán la similaridad existente entre items y usuario. Dado el ejemplo mostrado en la imagen anterior, podemos representar en dos dimensiones a los usuarios y los items, pudiendo apreciar como (por lo general) la proximidad entre los usuarios e items esta ligado al valor del voto:


<img src="./../imgs/03_02_01_02_CF_mf.png" style="width: 300px;"/>


* Como hemos podido apreciar, para crear el modelo (las matrices $P$ y $Q$), tenemos que definir previamente el parámetro ***k*** que será necesario "tunear" con el fin de ajustar el modelo a cada dataset. Este parámetro ***k*** representa el número de factores latentes de nuestro modelo.


<hr>


## <a name="M2">2.- Probabilistic Matrix Factorization: Cálculo de las Matrices P y Q</a>



* Desarrollando la expresión $R \approx P \cdot Q$, podemos inferir que la predicción de voto de un usuario $u$ a un item $i$ queda como:


$$\hat{r}_{u,i} = \vec{p}_u \cdot \vec{q}_i$$


* Representado de manera matricial seria:


$$\begin{bmatrix}
\hat{r}_{1,1} & \cdots  &  \hat{r}_{1,m} \\ 
\vdots  & \ddots  & \vdots  \\ 
\hat{r}_{1,n} & \cdots  &  \hat{r}_{n,m} 
\end{bmatrix} = 
\begin{bmatrix}
p_{1,1} & \cdots  &p_{1,k} \\ 
\vdots  & \ddots  & \vdots \\ 
p_{n,1} & \cdots  & p_{n,k}
\end{bmatrix}
\cdot
\begin{bmatrix}
q_{1,1} & \cdots  & q_{1,m}\\ 
\vdots  & \ddots  & \vdots \\ 
q_{k,1} & \cdots  & q_{k,m}
\end{bmatrix}$$


* Dónde $\vec{p}_u$ representa un vector fila de la matriz $P$ con los factores latentes del usuario $u$ y $\vec{q}_i$ representa un vector columna de la matriz $Q$ con los factores latentes del item $i$.


* Por lo tanto, podemos plantear la búsqueda de los factores latentes como un problema de optimización, en el cual buscamos minimizar el error cometido en los votos conocidos:


$$\min_{p,q} \sum_{(u,i) \in R} ( r_{u,i} - \vec{p}_u \cdot \vec{q}_i)^2$$


* Expresión a la que podemos añadir una regularización (L2) para evitar el *overfitting*:


$$\min_{p,q} \sum_{(u,i) \in R} ( r_{u,i} - \vec{p}_u \cdot \vec{q}_i)^2 + \lambda (||\vec{p}_u||^2 + ||\vec{q}_i||^2)$$


* Es posible resolver este problema mediante la técnica de descenso de gradiente, para lo cual debemos encontrar la derivada de la expresión anterior respecto del $\vec{p}_u$ y $\vec{q}_i$. Al hacerlo obtenemos las siguientes ecuaciones de actualización:


$$e_{u,i} = r_{u,i} - \vec{p}_u \cdot \vec{q}_i$$


$$\vec{p}_u = \vec{p}_u + \gamma (e_{u,i} \cdot \vec{q}_i - \lambda \vec{p}_u)$$


$$\vec{q}_i = \vec{q}_i + \gamma (e_{u,i} \cdot \vec{p}_u - \lambda \vec{q}_i)$$


* Donde $\lambda$ y $\gamma$ son dos hiperparámetros del modelo:
    + $\lambda$: Learning Rate
    + $\gamma$: indicará el peso que le damos al termino de complejidad del modelo (Regularización)


* Una vez entrenado el modelo, las matrices $P$ y $Q$ son aprendidas y no necesitan modificarse hasta que la matriz de votaciones cambie sustancialmente. Obtener una predicción una vez el modelo ha aprendido implica, simplemente, realizar el producto escalar de dos vectores de dimensión *k*, que, por lo general, suele ser un valor pequeño.


* A este algoritmo se le conoce como ***Probabilistic Matrix Factorization (PMF)***.



<hr>



## <a name="M3">3.- Probabilistic Matrix Factorization: Cálculo de las Matrices P y Q, añadiendo los bias</a>


* El ***modelo descrito anteriormente*** mejora significativamente la escalabilidad del filtrado colaborativo y, además, incremente notablemente la calidad de las predicciones y recomendaciones. Sin embargo, dicho modelo ***no se ajusta a la realidad puesto que no refleja los sesgos que los usuarios tienen cuando realizan votaciones***.


* Parece evidente pensar que no todos los usuarios tienen la misma interpretación de las votaciones. Por ejemplo, existen usuarios más "generosos" con las votaciones que tienden a asignar siempre valoraciones altas y existen usuarios más "tacaños" con las votaciones que tienden a asignar siempre valoraciones más bajas. Que el primer usuario valore un item con 5 y el segundo usuario valore el mismo item con un 4 no quiere decir que al primero le haya gustado más el item. Cada usuario hace su propia interpretación de lo que significan los votos 4 y 5.


* Igualmente, existen determinados items que socialmente tienen que gustar y existen otros items que está "mal visto" que gusten. Por ejemplo; si hablamos de un sistema de recomendación de películas, resulta extraño que alguien pueda otorgar la nota mínima a *El Padrino* aunque no le haya gustado. La presión social hace que dicha película sea importante, y eso condiciona nuestro voto sobre la misma. Igualmente, resulta extraño que alguien pueda otorgar la nota máxima a *Sharknado* ya que, socialmente, es considerada una película "mala".


* Para reflejar este fenómeno dentro de nuestro modelo de factorización matricial, debemos hacer algunas modificaciones sobre el mismo. Para empezar, cambiaremos cómo se calculan las predicciones:


$$\hat{r}_{u,i} = \mu + b_u + b_i + \vec{p}_u \cdot \vec{q}_i$$


* Donde:
    + $\mu$ representa la votación media de la base de datos
    + $b_u$ representa el bias (sesgo) del usuario $u$
    + $b_i$ representa el bias (sesgo) del item $i$
    + $\vec{p}_u \cdot \vec{q}_i$ simboliza la interacción entre el usuario $u$ y el item $i$. 


* De este modo, la predicción será calculada como la media de la base de datos
    + "*+/-*" un ajuste en función de cómo suele vota el usuario
    + "*+/-*" un ajuste de cómo suele votarse el item
    + "*+/-*" la interacción entre el usuario y el item


* Debido a este cambio, la función a minimizar es ahora la siguiente:


$$\min_{p,q} \sum_{(u,i) \in R} ( r_{u,i} - \mu - b_u - b_i - \vec{p}_u \cdot \vec{q}_i)^2 + \lambda (||\vec{p}_u||^2 + ||\vec{q}_i||^2 + b_u^2 + b_i^2)$$


* A la que, tras aplicar la derivada respecto de $b_u$, $q_i$, $\vec{p}_u$ y $\vec{q}_i$ obtenemos:


$$e_{u,i} = r_{u,i} - \mu - b_u - b_i - \vec{p}_u \cdot \vec{q}_i$$

$$b_u = b_u + \gamma (e_{u,i} - \lambda b_u)$$

$$b_i = b_i + \gamma (e_{u,i} - \lambda b_i)$$

$$\vec{p}_u = \vec{p}_u + \gamma (e_{u,i} \cdot \vec{q}_i - \lambda \vec{p}_u)$$

$$\vec{q}_i = \vec{q}_i + \gamma (e_{u,i} \cdot \vec{p}_u - \lambda \vec{q}_i)$$


<hr>


## <a name="M4">4.- Implementación y Ejemplo: PMF Cálculo de las Matrices P y Q</a>


* A continuación vamos a realizar un ejemplo con fines didácticos en el que vamos a ir implementando paso por paso todo lo visto anteriormente que será todo lo necesario para construir un Sistema de Recomendación basado en Filtrado Colaborativo, usando la técnica de la Factorización Matricial.


* Para ello seguiremos los siguientes pasos:

<span></span><br>
    4.1. [Lectura del Dataset (Matriz de Votos)](#M41)
<span></span><br>
    4.2. [Generación de las matrices de factores latentes de Usuarios e Items](#M42)
<span></span><br>
    4.3. [Implementación del cálculo de las predicciones](#M43)
<span></span><br>
    4.4. [Implementación del cálculo de las matrices P y Q](#M44)
<span></span><br>
    4.5. [Cálculo de las predicciones](#M45)
<span></span><br>
    4.6. [Evaluación del Sistema de Recomendación](#M46)


##### NOTA: Dado que los ejemplos que a continuación se muestran tienen fines didácticos, el objetivo es que se entiendan los algoritmos que permiten obtener las predicciones de los votos y la evaluación del sistema de recomendación. Las implementaciones que a continuación se propone no son ni mucho menos las óptimas pero si que intenta ser lo más sencillas y entendibles posible.


### <a name="M41">4.1.- Lectura del Dataset (Matriz de Votos)</a>


* En primer lugar lo que necesitamos es un Dataset en el que se encuentren las votaciones que han realizado los usuarios sobre una serie de items.


* Para realiza este ejemplo con fines didácticos vamos a utilizar un Dataset con las siguientes características:
    + ***6 Usuarios***
    + ***12 Items***
    + ***Votos del 1 al 5***: Los usuarios pueden votar los items con las notas $\{ 1, 2, 3, 4, 5 \}$
    
    
* Con este Dataset vamos a ser capaces de construir una matriz de votos en el que las filas representarán a los usuarios y las columnas a los items.


* Vamos a implementar una función que dado un fichero con la estructura '*id_user::id_movie::rating*' nos devuelva la matriz de votos (una lista de listas), donde es las filas encontramos a los usuarios y en las columnas las películas. La ausencia de voto se representará con un None.

In [1]:
# Definimos dos constantes con el número de usuarios e items
NUM_USERS = 6
NUM_ITEMS = 12

def read_ratings_matrix(file):
    ratings = [[None for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)] 
    
    with open(file, 'r') as reader:
        for line in reader:
            [u, i, rating] = line.split("::")
            ratings[int(u)][int(i)] = int(rating)
            
    return ratings

* Vamos a guardar la matriz de votos para utilizarla posteriormente y vamos a imprimirla por pantalla para ver la forma que tiene:

In [2]:
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

# Leemos el fichero y lo pasamos a una matriz (lista de listas)
RATINGS_FILE = './../data/db_prueba2.txt'
ratings_matrix = read_ratings_matrix(file=RATINGS_FILE)

# Mostramos la matriz de votos a modo informativo
pd.DataFrame(data=np.array([np.array(xi) for xi in ratings_matrix]),
             index=["U{}".format(str(i)) for i in range(NUM_USERS)],
             columns=["I{}".format(str(i)) for i in range(NUM_ITEMS)])

,I0,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11
U0,1,2,None,None,2,None,3,4,None,4,1,None
U1,None,None,1,5,None,5,3,1,None,5,2,1
U2,1,None,None,2,None,1,None,3,4,None,None,None
U3,None,1,4,4,None,None,3,None,5,4,None,1
U4,2,None,5,None,1,None,1,None,None,None,2,1
U5,None,None,5,2,1,None,None,4,None,1,None,2


### <a name="M42">4.2.- Generación de las matrices de factores latentes de Usuarios e Items</a>


* Recordemos que el objetivo que persigue la factorización matricial es el de calcular los factores latentes de los Usuarios ($P$) y de los Items ($Q$) partiendo de la matriz de votos:

$$R \approx P \cdot Q$$


* Con el objetivo de que al multiplicar los factores latentes del usuario con los factores lantentes del item, obtengamos la predicción de votos:

$$\hat{r}_{u,i} = \vec{p}_u \cdot \vec{q}_i$$


* Representado de manera matricial seria:


$$\begin{bmatrix}
\hat{r}_{1,1} & \cdots  &  \hat{r}_{1,m} \\ 
\vdots  & \ddots  & \vdots  \\ 
\hat{r}_{1,n} & \cdots  &  \hat{r}_{n,m} 
\end{bmatrix} = 
\begin{bmatrix}
p_{1,1} & \cdots  &p_{1,k} \\ 
\vdots  & \ddots  & \vdots \\ 
p_{n,1} & \cdots  & p_{n,k}
\end{bmatrix}
\cdot
\begin{bmatrix}
q_{1,1} & \cdots  & q_{1,m}\\ 
\vdots  & \ddots  & \vdots \\ 
q_{k,1} & \cdots  & q_{k,m}
\end{bmatrix}$$


* Por tanto vamos a crearnos las matrices $P$ y $Q$ inicializando los 'K' factores latentes de manera aleatoria, pudiendo utilizar también otras estrategias de inicialización.


##### NOTA: Dado que es un ejemplo didáctico y por simplicidad del mismo, vamos a utilizar 3 factores latentes. Para ver las diferencias tanto en rendimiento como en resultados se puede modificar este valor cambiándolo en la constante.

In [3]:
import random 

NUM_FACTORS = 3

p = [[random.random() for _ in range(NUM_FACTORS)] for _ in range(NUM_USERS)] 
q = [[random.random() for _ in range(NUM_FACTORS)] for _ in range(NUM_ITEMS)] 

* Mostremos a continuación como son estas matrices de factores latentes y como cambiarán sus valores tras la ejecución de la factorización matricial:

    + Factores latentes de los usuarios:

In [4]:
# Mostramos la matriz de Factores Latentes de los Usuarios
pd.DataFrame(data=np.array([np.array(xi) for xi in p]),
             index=["U{}".format(str(i)) for i in range(NUM_USERS)],
             columns=["K{}".format(str(i)) for i in range(NUM_FACTORS)])

,K0,K1,K2
U0,0.85,0.92,0.09
U1,0.62,0.77,0.17
U2,0.68,0.32,0.02
U3,0.94,0.91,0.98
U4,0.50,0.94,0.13
U5,0.71,0.32,0.04


In [5]:
# Mostramos la matriz de Factores Latentes de los Items
pd.DataFrame(data=np.array([np.array(xi) for xi in q]).T,
             index=["K{}".format(str(i)) for i in range(NUM_FACTORS)],
             columns=["I{}".format(str(i)) for i in range(NUM_ITEMS)])

,I0,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11
K0,0.45,0.04,0.32,0.54,0.85,0.68,0.74,0.44,0.56,0.10,0.88,0.18
K1,0.76,0.91,0.98,0.50,0.13,0.92,0.22,0.91,0.67,0.75,0.24,0.36
K2,0.69,0.41,0.44,0.49,0.96,0.20,0.16,0.39,0.94,0.59,0.45,0.17


### <a name="M43">4.3.- Implementación del cálculo de las predicciones</a>


* Uno de los puntos fuertes que tiene la factorización matricial es la simplicidad que tiene el cálculo de las predicciones ya que únicamente consiste en multiplicar las matrices de factores latentes de los usuarios y de los items, realizando esta multiplicación con la matriz traspuesta de los items:


$$\hat{R} = P \cdot {Q}'$$


* Para ello vamos a crear 2 funciones:
<span></span><br><br>
    + ***compute_predictions(p_u, q_i)***: función que dados los factores latentes de un usuario y de un item calcula la predicción.
<span></span><br><br>
    + ***calculate_predictions(p, q)***: función que dadas las matrices de factores latentes de los usuarios y de los items y haciendo uso de la función anteriore, calcula todas las predicciones del sistema.
    
    
##### NOTA: se implementa la función "compute_predictions(p_u, q_i)" para su uso posterior a la hora de calcular las matrices P y Q, de ahí que no se haya implementado en una única función el producto matricial de estas dos matrices.

In [6]:
from tqdm.notebook import tqdm

# Cálculo de una predicción
def compute_prediction (p_u, q_i):
    
    prediction = 0

    for k in range(NUM_FACTORS):
        prediction += p_u[k] * q_i[k]
    
    return prediction


# Calculo de todas las predicciones del sistema
def calculate_predictions(p, q):
    
    predictions = [[0 for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)] 
    
    # Recorremos la matriz de votos
    for i in tqdm(range(NUM_USERS), leave=False):
        for j in range(NUM_ITEMS):
            predictions[i][j] = compute_prediction(p_u=p[i], q_i=q[j])
    
    return predictions

* Con el fin de ver el correcto funcionamiento de esta funciones y dado que tenemos las matrices P y Q inicializadas, podemos realizar el cálculo de las predicciones, aunque estas serán unas predicciones erroneas:

In [7]:
predictions_matrix = calculate_predictions(p=p, q=q)

# Mostramos la matriz de Factores Latentes de los Items
pd.DataFrame(data=np.array([np.array(xi) for xi in predictions_matrix]),
             index=["U{}".format(str(i)) for i in range(NUM_USERS)],
             columns=["I{}".format(str(i)) for i in range(NUM_ITEMS)])

,I0,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11
U0,1.15,0.91,1.22,0.96,0.93,1.45,0.84,1.25,1.18,0.83,1.01,0.50
U1,0.98,0.79,1.02,0.80,0.79,1.16,0.65,1.03,1.02,0.74,0.80,0.41
U2,0.56,0.32,0.54,0.54,0.64,0.76,0.58,0.60,0.62,0.32,0.69,0.24
U3,1.79,1.27,1.62,1.44,1.86,1.67,1.04,1.62,2.06,1.35,1.49,0.66
U4,1.03,0.93,1.14,0.80,0.68,1.23,0.59,1.13,1.04,0.83,0.73,0.45
U5,0.59,0.33,0.56,0.56,0.68,0.79,0.60,0.62,0.65,0.33,0.72,0.25


### <a name="M44">4.4.- Implementación del cálculo de las matrices P y Q</a>



* Recordemos que el objetivo de la factorización matricial consiste en encontrar los factores latentes de los usuarios y de los items para poder calcular las predicciones; por tanto, nos encontramos con un problema de optimización que consiste en encontrar el valor de todos estos factores latentes minimizando el error cometido en los votos conocidos (añadida en la siguiente fórmula el factor de regularización L2):


$$\min_{p,q} \sum_{(u,i) \in R} ( r_{u,i} - \vec{p}_u \cdot \vec{q}_i)^2 + \lambda (||\vec{p}_u||^2 + ||\vec{q}_i||^2)$$


* Por tanto, podemos resolver este problema de optimización con la técnica de descenso de gradiente, para lo cual debemos encontrar la derivada de la expresión anterior respecto del $\vec{p}_u$ y $\vec{q}_i$, obteniendo las siguientes ecuaciones de actualización:


$$e_{u,i} = r_{u,i} - \vec{p}_u \cdot \vec{q}_i$$


$$\vec{p}_u = \vec{p}_u + \gamma (e_{u,i} \cdot \vec{q}_i - \lambda \vec{p}_u)$$


$$\vec{q}_i = \vec{q}_i + \gamma (e_{u,i} \cdot \vec{p}_u - \lambda \vec{q}_i)$$


* A continuación vamos a implementar una función llamada ***pmf()*** que recibiendo los parámetros que describimos a continuación, nos devolvera las matrices P y Q; es decir el modelo, con los factores latentes de los usuarios y de los items optimizados:
<span></span><br><br>
    + $p$: Matriz con los factores latentes de los usuarios
    + $q$: Matriz con los factores latentes de los items
    + $epochs$: Número de epochs a realizar por el gradiente descendente
    + $learning\_rate$: Factor de aprendizaje
    + $regularization\_rate$: Factor de Regularización

In [8]:
def pmf(p, q, epochs, learning_rate, regularization_rate):
    
    for epoch in tqdm(range(epochs), desc='Epoch', leave=False):
        for u in range(NUM_USERS):
            for i in range(NUM_ITEMS):
                if ratings_matrix[u][i] != None:
                    prediction = compute_prediction(p[u], q[i])
                    rating = ratings_matrix[u][i]
                    error = rating - prediction

                    for k in range(NUM_FACTORS):
                        p[u][k] += learning_rate * (error * q[i][k] - regularization_rate * p[u][k])
                        q[i][k] += learning_rate * (error * p[u][k] - regularization_rate * q[i][k])

    return p, q


# Definimos los Hiperparámetros
EPOCHS = 10
LEARNING_RATE = 0.05 # gamma
REGULARIZATION_RATE = 0.1 # lambda

# Ajustamos las matrices p y q
p, q = pmf(p=p, q=q, epochs=EPOCHS, learning_rate=LEARNING_RATE, regularization_rate=REGULARIZATION_RATE)

### <a name="M45">4.5.- Cálculo de las predicciones</a>


* A continuación realizamos el cálculo de las predicciones con las matriz $P$ y $Q$ ya ajustadas.

In [9]:
predictions_matrix = calculate_predictions(p=p, q=q)

# Mostramos la matriz de Factores Latentes de los Items
pd.DataFrame(data=np.array([np.array(xi) for xi in predictions_matrix]),
             index=["U{}".format(str(i)) for i in range(NUM_USERS)],
             columns=["I{}".format(str(i)) for i in range(NUM_ITEMS)])

,I0,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11
U0,1.52,1.58,4.72,3.70,1.65,4.36,2.45,3.69,4.74,3.67,1.76,1.47
U1,1.39,0.96,3.35,3.80,2.20,3.74,2.67,2.82,4.83,3.61,1.91,1.10
U2,1.01,0.83,2.60,2.53,1.36,2.62,1.67,2.08,3.27,2.51,1.24,0.80
U3,1.53,1.25,3.87,3.77,2.03,3.88,2.46,3.08,4.88,3.77,1.83,1.18
U4,1.50,1.72,4.61,3.05,1.16,3.77,1.71,3.38,4.05,3.33,1.35,1.30
U5,0.92,1.27,3.71,2.31,0.73,3.23,1.61,2.89,2.87,2.21,1.07,1.20


* A modo informativo y para una comparación visual (al tratarse este de un ejemplo didáctico) mostramos la matriz de votos:

In [10]:
# Mostramos la matriz de votos a modo informativo
pd.DataFrame(data=np.array([np.array(xi) for xi in ratings_matrix]),
             index=["U{}".format(str(i)) for i in range(NUM_USERS)],
             columns=["I{}".format(str(i)) for i in range(NUM_ITEMS)])

,I0,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11
U0,1,2,None,None,2,None,3,4,None,4,1,None
U1,None,None,1,5,None,5,3,1,None,5,2,1
U2,1,None,None,2,None,1,None,3,4,None,None,None
U3,None,1,4,4,None,None,3,None,5,4,None,1
U4,2,None,5,None,1,None,1,None,None,None,2,1
U5,None,None,5,2,1,None,None,4,None,1,None,2


* De la misma manera mostramos de nuevo las matrices $P$ y $Q$ para ver como han cambiado los factores latentes:

In [11]:
# Mostramos la matriz de Factores Latentes de los Usuarios
pd.DataFrame(data=np.array([np.array(xi) for xi in p]),
             index=["U{}".format(str(i)) for i in range(NUM_USERS)],
             columns=["K{}".format(str(i)) for i in range(NUM_FACTORS)])

,K0,K1,K2
U0,1.18,1.53,0.51
U1,1.46,0.74,1.04
U2,0.82,0.70,0.68
U3,1.18,1.06,1.05
U4,0.54,1.71,0.53
U5,0.83,1.36,-0.21


In [12]:
# Mostramos la matriz de Factores Latentes de los Items
pd.DataFrame(data=np.array([np.array(xi) for xi in q]).T,
             index=["K{}".format(str(i)) for i in range(NUM_FACTORS)],
             columns=["I{}".format(str(i)) for i in range(NUM_ITEMS)])

,I0,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11
K0,0.22,0.05,0.79,1.33,0.83,1.37,1.28,0.92,1.50,0.91,0.76,0.42
K1,0.63,0.93,2.31,1.05,0.16,1.62,0.47,1.61,1.43,1.26,0.40,0.63
K2,0.58,0.20,0.47,1.04,0.83,0.52,0.43,0.28,1.52,1.28,0.49,0.02


### <a name="M46">4.6.- Evaluación del Sistema de Recomendación</a>


* Por último vamos a calcular el error medio (MAE) cometido en las predicciones realizadas.


* Al no dividir en este ejemplo dos datos en conjunto de entrenamiento y test, vamos a evaluar las predicciones sobre el dataset del ejemplo.


* Para ello vamos a implementar una función llamada ***get_mae()*** que recibirá como parámetros la matriz de votos y la matriz de predicciones y devolverá el MAE del Sistema de Recomendación.


* Recordar que definimos el MAE de un usuario como:
<span></span><br><br>
$$MAE_u = \frac{ \sum_{i \in I^T_u} \mid r_{u,i} - \hat{r}_{u,i} \mid  }{\#I^T_u} $$
<span></span><br><br>
    donde $I^T_u$ representa el conjunto de items de test votados por el usuario $u$.
<span></span><br><br>
* Definimos el *MAE* del sistema como el promedio del *MAE* de cada usuario:
<span></span><br><br>
$$MAE = \frac{ \sum_{u \in U^T} MAE_u }{ \#U^T } $$

In [13]:
def get_mae(ratings_matrix, predictions_matrix):
    
    mae_users = [None for _ in ratings_matrix]
    
    # Recorremos la matriz de votos
    for i, u in enumerate(tqdm(ratings_matrix, leave=False)):
        # Calculamos el MAE de cada usuario
        sum_user = 0
        count = 0
        for j, v in enumerate(ratings_matrix[0]):
            if ratings_matrix[i][j] != None and predictions_matrix[i][j] != None:
                sum_user += abs(ratings_matrix[i][j] - predictions_matrix[i][j])
                count += 1
        mae_users[i] = sum_user/count if count > 0 else None

    return np.nanmean(np.array(mae_users, dtype=np.float), axis=0)


mae = get_mae(ratings_matrix=ratings_matrix, predictions_matrix=predictions_matrix)

print('MAE del sistema: {:0.4f}'.format(mae))

MAE del sistema: 0.6362


#### NOTA: Una vez realizada la implementación se recomienda ver como cambian las recomendaciones y como mejora o empeora el MAE del Sistema de Recomendación modificando los Hiperparámetros del algoritmo: Número de factores, Número de epochs, Learning Rate, Regularization Rate. 

<hr>


## <a name="M5">5.- Implementación y Ejemplo: PMF Cálculo de las Matrices P y Q, añadiendo los bias</a>


* A continuación vamos a realizar un ejemplo con fines didácticos en el que vamos a ir implementando paso por paso todo lo visto anteriormente que será todo lo necesario para construir un Sistema de Recomendación basado en Filtrado Colaborativo, usando la técnica de la Factorización Matricial, añadiendo el bias.


* Recordemos que para realizar esta implementación, la predicción se calcula como:


$$\hat{r}_{u,i} = \mu + b_u + b_i + \vec{p}_u \cdot \vec{q}_i$$


* Donde:
    + $\mu$ representa la votación media de la base de datos
    + $b_u$ representa el bias (sesgo) del usuario $u$
    + $b_i$ representa el bias (sesgo) del item $i$
    + $\vec{p}_u \cdot \vec{q}_i$ simboliza la interacción entre el usuario $u$ y el item $i$


* La función a minimizar es ahora la siguiente:


$$\min_{p,q} \sum_{(u,i) \in R} ( r_{u,i} - \mu - b_u - b_i - \vec{p}_u \cdot \vec{q}_i)^2 + \lambda (||\vec{p}_u||^2 + ||\vec{q}_i||^2 + b_u^2 + b_i^2)$$


* Aplicando las derivadas respecto de $b_u$, $q_i$, $\vec{p}_u$ y $\vec{q}_i$ obtenemos:

$$e_{u,i} = r_{u,i} - \mu - b_u - b_i - \vec{p}_u \cdot \vec{q}_i$$

$$b_u = b_u + \gamma (e_{u,i} - \lambda b_u)$$

$$b_i = b_i + \gamma (e_{u,i} - \lambda b_i)$$

$$\vec{p}_u = \vec{p}_u + \gamma (e_{u,i} \cdot \vec{q}_i - \lambda \vec{p}_u)$$

$$\vec{q}_i = \vec{q}_i + \gamma (e_{u,i} \cdot \vec{p}_u - \lambda \vec{q}_i)$$




* Para ello seguiremos los siguientes pasos

<span></span><br>
    5.1. [Lectura del Dataset (Matriz de Votos) - (biased)](#M51)
<span></span><br>
    5.2. [Generación de las matrices de factores latentes de Usuarios e Items, así como un array de los bias de los usuarios y los items - (biased)](#M52)
<span></span><br>
    5.3. [Implementación del cálculo de las predicciones - (biased)](#M53)
<span></span><br>
    5.4. [Implementación del cálculo de las matrices P y Q, añadiendo el bias - (biased)](#M54)
<span></span><br>
    5.5. [Cálculo de las predicciones - (biased)](#M55)
<span></span><br>
    5.6. [Evaluación del Sistema de Recomendación - (biased)](#M56)



### <a name="M51">5.1.- Lectura del Dataset (Matriz de Votos) - (biased)</a>


* Este punto ya esta realizado en el punto anterior "4.1. [Lectura del Dataset (Matriz de Votos)](#M41)"


### <a name="M52">5.2.- Generación de las matrices de factores latentes de Usuarios e Items, así como un array de los bias de los usuarios y los items - (biased)</a>


In [14]:
import random 

NUM_FACTORS = 3

p = [[random.random() for _ in range(NUM_FACTORS)] for _ in range(NUM_USERS)] 
q = [[random.random() for _ in range(NUM_FACTORS)] for _ in range(NUM_ITEMS)] 

bu = [random.random() for _ in range(NUM_USERS)]
bi = [random.random() for _ in range(NUM_ITEMS)]

* A modo de ejemplo, mostramos el contenido de los bias y de los factores latentes de los usuarios y de los items con el fin de compararlos una vez ajustadas estas dos matrices:

In [15]:
# Mostramos los bias de los Usuarios
pd.DataFrame(data=np.array(bu),
             index=["U{}".format(str(i)) for i in range(NUM_USERS)],
             columns=["bu"]).transpose()

,U0,U1,U2,U3,U4,U5
bu,0.48,0.14,0.50,0.46,0.23,0.21


In [16]:
# Mostramos los bias de los Items
pd.DataFrame(data=np.array(bi),
             index=["U{}".format(str(i)) for i in range(NUM_ITEMS)],
             columns=["bu"]).transpose()

,U0,U1,U2,U3,U4,U5,U6,U7,U8,U9,U10,U11
bu,0.58,0.88,0.22,0.80,0.20,0.99,0.81,0.84,0.17,0.26,0.36,0.69


In [17]:
# Mostramos la matriz de Factores Latentes de los Usuarios
pd.DataFrame(data=np.array([np.array(xi) for xi in p]),
             index=["U{}".format(str(i)) for i in range(NUM_USERS)],
             columns=["K{}".format(str(i)) for i in range(NUM_FACTORS)])

,K0,K1,K2
U0,0.65,0.80,0.08
U1,0.49,0.26,0.02
U2,0.62,0.46,0.01
U3,0.66,0.85,0.11
U4,0.00,0.41,0.74
U5,0.02,0.69,0.52


In [18]:
# Mostramos la matriz de Factores Latentes de los Items
pd.DataFrame(data=np.array([np.array(xi) for xi in q]).T,
             index=["K{}".format(str(i)) for i in range(NUM_FACTORS)],
             columns=["I{}".format(str(i)) for i in range(NUM_ITEMS)])

,I0,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11
K0,0.59,0.80,0.51,0.86,0.98,0.13,0.60,0.56,0.71,0.17,0.91,0.25
K1,0.08,0.14,0.65,0.90,0.36,0.21,0.57,0.04,0.67,0.36,0.52,0.66
K2,0.50,0.29,0.25,0.62,0.53,0.37,0.04,0.94,0.55,0.32,0.75,0.93


### <a name="M53">5.3.- Implementación del cálculo de las predicciones - (biased)</a>


* El cálculo de las predicciones se realiza como:


$$\hat{r}_{u,i} = \mu + b_u + b_i + \vec{p}_u \cdot \vec{q}_i$$


* Para ello vamos a crear 2 funciones:
<span></span><br><br>
    + ***compute_biased_prediction(avg, b_u, b_i, p_u, q_i)***: función que dado la media de la base de datos, el bias del usuario y del items y los factores latentes de un usuario y de un item calcula la predicción.
<span></span><br><br>
    + ***calculate_biased_predictions(avg, bu, bi, p, q)***: función que dado la media de la base de datos, los bias de los usuarios y de los items y los factores latentes de los usuarios y de los items y haciendo uso de la función anterior, calcula todas las predicciones del sistema.
    
    
* Previo a estas funciones, implementaremos una función que nos devuelva el voto medio de la base de datos:

In [19]:
# Calculo del voto medio de la base de datos
def get_avg_ratings(ratings_matrix):
    
    sum_ratings = 0
    count_ratings = 0

    for u in tqdm(range(NUM_USERS), leave=False):
        for i in range(NUM_ITEMS):
            if ratings_matrix[u][i] != None:
                sum_ratings += ratings_matrix[u][i]
                count_ratings += 1

    return sum_ratings/float(count_ratings)  


# Cálculo de una predicción
def compute_biased_prediction (avg, b_u, b_i, p_u, q_i):
    deviation = 0
    for k in range(NUM_FACTORS):
        deviation += p_u[k] * q_i[k]
        
    prediction = avg + b_u + b_i + deviation
    return prediction


# Calculo de todas las predicciones del sistema
def calculate_biased_predictions(avg, bu, bi, p, q):
    
    predictions = [[0 for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)] 
    
    # Recorremos la matriz de votos
    for i in tqdm(range(NUM_USERS), leave=False):
        for j in range(NUM_ITEMS):
            predictions[i][j] = compute_biased_prediction(avg=avg, b_u=bu[i], b_i=bi[j], p_u=p[i], q_i=q[j])
    
    return predictions


* Con el fin de ver el correcto funcionamiento de esta funciones y dado que tenemos las matrices P y Q inicializadas, así como los bias de los usuarios y de los items, podemos realizar el cálculo de las predicciones, aunque estas serán unas predicciones erroneas:

In [20]:
# Calculamos el voto medio de la base de datos
avg = get_avg_ratings(ratings_matrix)
print('Voto medio de la base de datos = {:0.2f}'.format(avg))

# Calculamos las predicciones
prediction_matrix = calculate_biased_predictions(avg=avg, bu=bu, bi=bi, p=p, q=q)

# Mostramos la matriz de Factores Latentes de los Items
pd.DataFrame(data=np.array([np.array(xi) for xi in predictions_matrix]),
             index=["U{}".format(str(i)) for i in range(NUM_USERS)],
             columns=["I{}".format(str(i)) for i in range(NUM_ITEMS)])

Voto medio de la base de datos = 2.56


,I0,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11
U0,1.52,1.58,4.72,3.70,1.65,4.36,2.45,3.69,4.74,3.67,1.76,1.47
U1,1.39,0.96,3.35,3.80,2.20,3.74,2.67,2.82,4.83,3.61,1.91,1.10
U2,1.01,0.83,2.60,2.53,1.36,2.62,1.67,2.08,3.27,2.51,1.24,0.80
U3,1.53,1.25,3.87,3.77,2.03,3.88,2.46,3.08,4.88,3.77,1.83,1.18
U4,1.50,1.72,4.61,3.05,1.16,3.77,1.71,3.38,4.05,3.33,1.35,1.30
U5,0.92,1.27,3.71,2.31,0.73,3.23,1.61,2.89,2.87,2.21,1.07,1.20


### <a name="M54">5.4.- Implementación del cálculo de las matrices P y Q, añadiendo el bias - (biased)</a>


* Recordemos como calcular la predicción, el error y los parámetros a calcular:

$$\hat{r}_{u,i} = \mu + b_u + b_i + \vec{p}_u \cdot \vec{q}_i$$

$$e_{u,i} = r_{u,i} - \mu - b_u - b_i - \vec{p}_u \cdot \vec{q}_i$$

$$b_u = b_u + \gamma (e_{u,i} - \lambda b_u)$$

$$b_i = b_i + \gamma (e_{u,i} - \lambda b_i)$$

$$\vec{p}_u = \vec{p}_u + \gamma (e_{u,i} \cdot \vec{q}_i - \lambda \vec{p}_u)$$

$$\vec{q}_i = \vec{q}_i + \gamma (e_{u,i} \cdot \vec{p}_u - \lambda \vec{q}_i)$$


* A continuación vamos a implementar una función llamada ***biased_pmf()*** que recibiendo los parámetros que describimos a continuación, nos devolvera las matrices P y Q así como los bias de los usuarios y de los items; es decir el modelo, con los factores latentes de los usuarios y de los items y los bias optimizados:
<span></span><br><br>
    + $avg$: Voto medio de la base de datos
    + $bu$: Bias de los usuarios
    + $bi$: Bias de los items
    + $p$: Matriz con los factores latentes de los usuarios
    + $q$: Matriz con los factores latentes de los items
    + $epochs$: Número de epochs a realizar por el gradiente descendente
    + $learning\_rate$: Factor de aprendizaje
    + $regularization\_rate$: Factor de Regularización

In [21]:
def biased_pmf(avg, bu, bi, p, q, epochs, learning_rate, regularization_rate):
     
    for epoch in tqdm(range(epochs), desc='Epoch', leave=False):    
        for u in range(NUM_USERS):
            for i in range(NUM_ITEMS):
                if ratings_matrix[u][i] != None:
                    prediction = compute_biased_prediction(avg=avg, b_u=bu[u], b_i=bi[i], p_u=p[u], q_i=q[i])
                    rating = ratings_matrix[u][i]
                    error = rating - prediction
                    
                    for k in range(NUM_FACTORS):
                        p[u][k] += learning_rate * (error * q[i][k] - regularization_rate * p[u][k])
                        q[i][k] += learning_rate * (error * p[u][k] - regularization_rate * q[i][k])
                    
                    bu[u] += learning_rate * (error - regularization_rate * bu[u])
                    bi[i] += learning_rate * (error - regularization_rate * bi[i])

    return bu, bi, p, q


# Definimos los Hiperparámetros
EPOCHS = 10
LEARNING_RATE = 0.05 # gamma
REGULARIZATION_RATE = 0.1 # lambda

# Calculamos el voto medio de la base de datos
avg = get_avg_ratings(ratings_matrix)

# Ajustamos las matrices p y q
bu, bi, p, q = biased_pmf(avg=avg, bu=bu, bi=bi, p=p, q=q, epochs=EPOCHS, 
                          learning_rate=LEARNING_RATE, regularization_rate=REGULARIZATION_RATE)

### <a name="M55">5.5.- Cálculo de las predicciones - (biased)</a>

* A continuación realizamos el cálculo de las predicciones con los bias $bu$ y $bi$ y las matriz $P$ y $Q$ ya ajustadas.

In [22]:
predictions_matrix = calculate_biased_predictions(avg=avg, bu=bu, bi=bi, p=p, q=q)

# Mostramos la matriz de Factores Latentes de los Items
pd.DataFrame(data=np.array([np.array(xi) for xi in predictions_matrix]),
             index=["U{}".format(str(i)) for i in range(NUM_USERS)],
             columns=["I{}".format(str(i)) for i in range(NUM_ITEMS)])

,I0,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11
U0,1.64,1.84,3.78,3.37,1.94,3.19,2.62,3.18,4.15,3.50,1.84,1.37
U1,1.84,2.47,1.85,4.01,2.76,3.46,3.31,2.12,3.98,4.51,1.91,1.07
U2,1.14,1.39,2.88,2.73,1.41,2.58,1.98,2.50,3.37,2.83,1.31,0.80
U3,1.43,1.61,3.97,3.86,2.21,3.16,2.80,3.20,4.81,3.88,2.00,1.19
U4,1.65,1.63,4.40,2.36,1.15,2.79,1.69,3.69,3.39,2.16,1.71,1.59
U5,1.82,1.75,4.60,2.06,0.98,2.80,1.49,3.93,3.11,1.79,1.75,1.80


* A modo informativo y para una comparación visual (al tratarse este de un ejemplo didáctico) mostramos la matriz de votos:

In [23]:
# Mostramos la matriz de votos a modo informativo
pd.DataFrame(data=np.array([np.array(xi) for xi in ratings_matrix]),
             index=["U{}".format(str(i)) for i in range(NUM_USERS)],
             columns=["I{}".format(str(i)) for i in range(NUM_ITEMS)])

,I0,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11
U0,1,2,None,None,2,None,3,4,None,4,1,None
U1,None,None,1,5,None,5,3,1,None,5,2,1
U2,1,None,None,2,None,1,None,3,4,None,None,None
U3,None,1,4,4,None,None,3,None,5,4,None,1
U4,2,None,5,None,1,None,1,None,None,None,2,1
U5,None,None,5,2,1,None,None,4,None,1,None,2


* De la misma manera, mostramos el contenido de los bias y de los factores latentes de los usuarios y de los items para que se pueda comprobar como han cambiado desde su inicialización aleatoria:

In [24]:
# Mostramos los bias de los Usuarios
pd.DataFrame(data=np.array(bu),
             index=["U{}".format(str(i)) for i in range(NUM_USERS)],
             columns=["bu"]).transpose()

,U0,U1,U2,U3,U4,U5
bu,-0.07,0.14,-0.72,-0.21,-0.49,-0.46


In [25]:
# Mostramos los bias de los Items
pd.DataFrame(data=np.array(bi),
             index=["U{}".format(str(i)) for i in range(NUM_ITEMS)],
             columns=["bu"]).transpose()

,U0,U1,U2,U3,U4,U5,U6,U7,U8,U9,U10,U11
bu,-0.40,-0.12,0.80,0.55,-0.50,0.77,-0.07,0.74,1.00,0.60,-0.45,-0.82


In [26]:
# Mostramos la matriz de Factores Latentes de los Usuarios
pd.DataFrame(data=np.array([np.array(xi) for xi in p]),
             index=["U{}".format(str(i)) for i in range(NUM_USERS)],
             columns=["K{}".format(str(i)) for i in range(NUM_FACTORS)])

,K0,K1,K2
U0,-0.00,0.67,-0.28
U1,0.75,-0.08,-1.08
U2,0.12,0.44,-0.22
U3,0.42,1.05,-0.30
U4,-0.52,0.57,0.54
U5,-0.76,0.43,0.77


In [27]:
# Mostramos la matriz de Factores Latentes de los Items
pd.DataFrame(data=np.array([np.array(xi) for xi in q]).T,
             index=["K{}".format(str(i)) for i in range(NUM_FACTORS)],
             columns=["I{}".format(str(i)) for i in range(NUM_ITEMS)])

,I0,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11
K0,0.23,0.36,-0.27,1.08,1.00,0.30,0.60,0.05,0.92,0.86,0.71,0.05
K1,-0.41,-0.63,1.24,0.48,-0.02,-0.02,0.19,0.43,1.10,0.37,0.04,-0.12
K2,0.61,0.40,1.25,0.02,0.17,0.21,-0.22,1.22,0.31,-0.54,0.80,0.79


### <a name="M56">5.6.- Evaluación del Sistema de Recomendación - (biased)</a>


* Este punto ya esta realizado en el punto anterior "4.6. [Evaluación del Sistema de Recomendación](#M46)"


In [28]:
mae = get_mae(ratings_matrix=ratings_matrix, predictions_matrix=predictions_matrix)

print('MAE del sistema: {:0.4f}'.format(mae))

MAE del sistema: 0.4669


#### NOTA: Una vez realizada la implementación se recomienda ver como cambian las recomendaciones y como mejora o empeora el MAE del Sistema de Recomendación modificando los Hiperparámetros del algoritmo: Número de factores, Número de epochs, Learning Rate, Regularization Rate. 

## Referencias

Mnih, A., & Salakhutdinov, R. R. (2008). **Probabilistic matrix factorization**. In Advances in neural information processing systems (pp. 1257-1264).

Koren, Y., Bell, R., & Volinsky, C. (2009). **Matrix factorization techniques for recommender systems**. Computer, (8), 30-37.

<hr>


# Bonus Track - Regularización


* En esta Bonus Track vamos a ver que es la Regularización y los 3 tipos de regularización más populares (LASSO, Ridge y ElasticNet):
<span></span><br>
    1. [Regularización](#MB1)
<span></span><br>
    2. [Regulariazión L1 - LASSO](#MB2)
<span></span><br>
    3. [Regulariazión L2 - Ridge](#MB3)
<span></span><br>
    4. [Regularización ElasticNet](#MB4)
<span></span><br>
    5. [¿Cuando y Qué tipo de regularización usar? - Resumen](#MB5)
    
    
<hr>


## <a name="MB1">1. Regularización</a>


* ***La Regularización en el Deep | Machine Learning es un método que permite a los Algoritmos de Aprendizaje construir modelos menos complejos con el fin de que estos generalizen mejor, reduciendo el sobreajuste (Overfitting) del modelo*** a los datos de entrenamiento.


* Por lo general los modelos más simples tienden a generalizar mejor que los modelos complejos; ya que estos últimos tienden a sobreajustarse a los datos de entrenamiento, obteniendo medidas de calidad (MSE, MAE, etc.) muy buenas en los datos de entrenamiento pero no tan buenas para los datos de test. Con modelos más simples somos capaces de obtener un equilibrio mayor de las medidas de calidad de los modelos entre los datos de entrenamiento y test.


* Para problemas de ***regresión*** lo que se pretende es ***encontrar un modelo de la forma $h(x) = \beta_0 + X_1 \cdot \beta_1 + ... + X_n \cdot \beta_n$***, obteniendo los mejores parámetros $\beta_j$ que mejor se ajusten a los datos de entrenamiento y esto lo conseguimos ***minimizando la función de perdida (J)*** (por lo general) con el Error Cuadrático Medio (MSE).
<span></span><br><br>
<span style="font-size:16px">$$J(\beta) = MSE = \frac{1}{N} \sum_{i=1}^{N} (h(x^{(i)}) - y^{(i)})^2$$</span>


* Lo que hacen los diferentes ***métodos de Regularización es penalizar la complejidad del modelos***, añadiendo a la función de perdida un nuevo término **'C'** que nos ***indicará la complejidad del modelo*** y este estará ***regulado por un hiperparámetro '$\lambda$' que indicará el peso que le damos al termino de complejidad***, quedando la función de perdida que queremos minimizar de la siguiente manera:
<span></span><br><br> 
<span style="font-size:16px">$$J(\beta) = MSE - \lambda \; C$$</span>


* En función de como definamos o midamos la complejidad, tendremos los distintos tipos de regularización, siendo los más comunes los siguientes métodos:

    + L1 - LASSO
    + L2 - Ridge
    + ElasticNet



<hr>


## <a name="MB2">2. Regulariazión L1 - LASSO</a>


* La Regularización L1; tambien conocida como LASSO (Least Absolute Shrinkage ans Selection Operator), se denomina L1 por utilizar como ***termino de complejidad la norma L1 que es la suma del valor absoluto de los parámetros del modelo***:
<span></span><br><br> 
<span style="font-size:16px">$$C = L1 = \left \| \beta\right \|_1 = \sum  \left | \beta_j \right |$$</span>


* Haciendo uso de la Regularización L1 en la regresión lineal, la función de perdida a minimizar quedaría de la siguiente manera:
<span></span><br><br>
<span style="font-size:16px">$$\underset{\beta}{min} \: J(\beta) = \frac{1}{2N} \sum_{i=1}^{N} (h(x) - y^{(i)})^2 + \lambda \; \sum_{j=1}^{j}  \left | \beta_j \right |$$</span>


* La Regularización L1 debemos usarla cuando sospechemos que ***varias de las variables de entrada vayan a ser irrelevantes para el modelo***, ya que estas tenderán a tener valores muy cercanos a cero.


* También debemos de usarlo cuando las ***variables no están muy correladas entre sí***.


* La Regularización L1 se propuso con el objetivo de dotar a los modelos (en particular a las redes neuronales) de la capacidad de 'olvidar', de ahí que con este tipo de Regularización obtengamos modelos en los que los parámetros de variables muy relevantes tengan valores altos y los parámetros sobre variables irrelevantes o altamente correladas con otras variables, tengan valores cercanos a cero.


* Cuando hay variables altamente correladas entre sí, la Regularización L1 tiende a seleccionar una de ellas de forma aleatoria y olvidar el resto, poniendo sus parámetros con valores muy cercanos a cero.

<hr>


## <a name="MB3">3. Regulariazión L2 - Ridge</a>


* La Regularización L2; tambien conocida como Rigde (Arista), se denomina L2 por utilizar como ***termino de complejidad la norma L2 al cuadrado de un vector (o norma euclidea), que es la suma de los valores al cuadrado de los parámetros del modelo***:
<span></span><br><br> 
<span style="font-size:16px">$$C = L2 = \left \| \beta\right \|_2^2 = \sum  \beta_j^2$$</span>


* Haciendo uso de la Regularización L2 en la regresión lineal, la función de perdida a minimizar quedaría de la siguiente manera:
<span></span><br><br>
<span style="font-size:16px">$$\underset{\beta}{min} \: J(\beta) = \frac{1}{2N} \sum_{i=1}^{N} (h(x) - y^{(i)})^2 + \lambda \; \sum_{j=1}^{j}  \beta_j^2$$</span>
    

* La Regularización L2 debemos usarla cuando en nuestro Dataset:
    + ***La mayoría de las variables sean relevantes***.
    + ***Las variables estén bastante correladas entre sí***. 


* La Regularización L2, penaliza fuertemente los valores de parámetros grandes y favorece la obtención de parámetros con valores pequeños, lo que hace que se minimice el efecto de la correlación entre las variables.

<hr>


## <a name="MB4">4. Regularización ElasticNet</a>


* La Regularización ElasticNet (Redes Elásticas) combina la Regularización L1 con la Regularización L2, calculando la ***complejidad del modelo como la suma de las complejidades dadas por la regularización L1 y L2***:
<span></span><br><br>
<span style="font-size:16px">$$C = L1 + L2 = \left \| \beta\right \|_1 + \left \| \beta\right \|_2^2 = \sum \left | \beta_j \right | + \sum  \beta_j^2$$</span>


* Por lo general en la Regularización ElasticNet se aplica un ***factor de importancia relativa 'r'*** para dar (si se quiere) más peso a la Regularización L1 que a la L2 o viceversa:
<span></span><br><br>
<span style="font-size:16px">$$C = r \; L1 + (1-r) \; L2 $$</span>


* Haciendo uso de la Regularización ElasticNet en la regresión lineal, la función de perdida a minimizar quedaría de la siguiente manera:
<span></span><br><br>
    <span style="font-size:16px">$$\underset{\beta}{min} \: J(\beta) = \frac{1}{2N} \sum_{i=1}^{N} (h(x) - y^{(i)})^2 + \lambda \; r \;\sum_{j=1}^{j}  \left | \beta_j \right | + \lambda \; (1-r) \; \sum_{j=1}^{j}  \beta_j^2$$</span>


* La Regularización ElasticNet debemos usarla cuando en nuestro ***Dataset tengamos un gran número de variables y cuando se cumplas las características descritas para ambas regularizaciones L1 y L2***, que son:
    + ***Varias de las variables de entrada vayan a ser irrelevantes***.
    + ***Las variables estén bastante correladas entre sí***.
    
<hr>


## <a name="MB5">5. ¿Cuando y Qué tipo de regularización usar? - Resumen</a>

### ¿Cuando hay que usar Regularización?


* La Regularización es un método que no resulta necesario aplicar cuando trabajamos en el ajuste de modelos "simples" con pocos parámetros; sin embargo, resulta imprescindible ***aplicarlo cuando se trata de ajustar un modelo muy complejo en el que hay cientos o miles de parámetros***.


### ¿Qué tipo de regularización usar? - Resumen


#### - L1 - LASSO


* ***Varias de las variables de entrada vayan a ser irrelevantes para el modelo***


* ***Las variables del Dataset no están muy correladas entre sí***.


#### - L2 -Ridge


* ***La mayoría de las variables del Dataset sean relevantes***.


* ***Las variables del Dataset estén bastante correladas entre sí***. 


#### - ElasticNet


* ***En nuestro Dataset tengamos un gran número de variables***.


* ***Varias de las variables de entrada vayan a ser irrelevantes***.


* ***Las variables del Dataset estén bastante correladas entre sí***.

<hr>


*Este documento ha sido desarrollado por **Ricardo Moya**, basandose en el material creado por **Fernando Ortega**. Dpto. Sistemas Informáticos, ETSI de Sistemas Informáticos, Universidad Politécnica de Madrid.* respetando la licencia: "Atribución-NoComercial-CompartirIgual" definida por **Creative Commons Corporation**.


<img src="../imgs/CC_BY-NC-SA.png" alt="CC BY-NC">

<p style="text-align:center"><b>Atribución-NoComercial-CompartirIgual</b></p>
